In [1]:
import DMCLP_Instance_Reader as reader
import DMCLP_Instance_Generator as generator
import DMCLP_Gurobi as grb
import DMCLP_LDA as lda
import DMCLP_CA as ca
import DMCLP_utils as utils
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [2]:
basepath = 'C:\\Users\\chich\\Google Drive\\Academic\\07.Codes\\Python Codes\\01_Aerial_Wireless_Communication\\02_INFORMS_JOC\\Publication\\Instances Test2\\'
#basepath = 'C:\\Users\\Abc\\Google Drive\\Academic\\07.Codes\\Python Codes\\01_Aerial_Wireless_Communication\\02_INFORMS_JOC\\Publication\\'

In [3]:
args_common = {'Q':{'x':[0,1500],'y':[0,1500],'h':[50,500]},
               'F':48.0855,
               'eta':2.5,
               'alpha':4.88,
               'beta':0.43,
               'B':0.1-21,
              }

args_common['L_min'] = args_common['F'] + 10*args_common['eta']*np.log10(min(args_common['Q']['h'])) + args_common['B']

args_lda = {'DC_iter_max':100,
            'DC_gradient_step_size':50,
            'DC_epsilon_gradient':0.1,
            'DC_number_of_replications':1,
            'LDA_halving_iter_max':10,
            'LDA_halving_tolerance':0.01,
            'LDA_epsilon_acc':0.01,
            'LDA_iter_max':1,
            'LDA_CPU_max':14400,
            'verbose':False,
           }

args_grb = {'time_limit':30,
            'show_console':False,
            'compute_IIS':False,
            'write_file':True,
            'write_solution':False,
           }

args_ca = {'w_bar':0.5,
           'delta_w':0.2,
           'd_bar':105,
           'delta_d':0.2,
           'delta_t':0.2,
           'CA_grid_division':10,
           'CA_h_division':10,
           'CA_P':0.9,
           'CA_rho':20,
           'CA_varrho':0.5,
           'CA_K':20,
           'CA_l':20,
           'CA_number_of_replications':10
          }

In [5]:
txt = '{:^20},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10},{:^10}\n'\
.format('Instance',
        'Period',
        'x_grb',
        'y_grb',
        'h_grb',
        'x_lda',
        'y_lda',
        'h_lda',
        'x_ca',
        'y_ca',
        'h_ca',
        'obj_grb',
        'obj_lda',
        'obj_ca',
        'cpu_grb',
        'cpu_lda',
        'cpu_ca')


trends = ['inc','dec','random']
number_of_replications = 2
number_of_grids = [4]
number_of_periods = [10]
for s in number_of_grids:
    for t in number_of_periods:
        for trend_w in trends:
            for trend_d in trends:
                for rep_no in range(number_of_replications):

                    # generate instance
                    I = range(s)
                    T = range(t)
                    upsilon_w = utils.upsilon(trend=trend_w)
                    upsilon_d = utils.upsilon(trend=trend_d)
                    Y,W,D,p = generator.generate(args_common['Q'],upsilon_w,upsilon_d,nbGrid=s,nbPeriod=t,seed=rep_no,
                                                w_bar=0.5,d_bar=105,delta_w=0.2,delta_d=0.2,delta_t=0.2,delta_p=0.2)
                    
                    # save instance to a txt file
                    with open('{}\\DMCLP_{}_{}_{}_{}_{}.txt'.format(basepath,s,t,trend_w,trend_d,rep_no),'w') as f:
                        f.write(utils.generate_save_file(Y,W,D,p))
                        f.close()

                    # solve instance with CA
                    args_for_ca = utils.merge_two_dicts(args_ca,args_common)
                    X_ca,cpu_ca = ca.solve(I,T,Y,W,D,p,upsilon_w,upsilon_d,args_for_ca)

                    # compute bigM value for the discrete formulation
                    d_min = np.Inf
                    for i in I:
                        for t in T:
                            if D[i][t] < d_min:
                                d_min = D[i][t]

                    args_common['bigM'] = int(max( (args_common['F'] + 10*args_common['eta']*np.log10(np.sqrt((max(args_common['Q']['x']) - min(args_common['Q']['x']))**2 + (max(args_common['Q']['y']) - min(args_common['Q']['y']))**2 + (max(args_common['Q']['h']))**2)) - d_min),
                                                  (args_common['F'] + 10*args_common['eta']*np.log10(np.sqrt((max(args_common['Q']['x']) - min(args_common['Q']['x']))**2 + (max(args_common['Q']['y']) - min(args_common['Q']['y']))**2 + (max(args_common['Q']['h']))**2)) - args_common['B'])
                                                 )
                                             ) + 1

                    # solve instance with LDA
                    args_for_lda = utils.merge_two_dicts(args_lda,args_common)
                    X_lda,cpu_lda = lda.solve(I,T,Y,W,D,p,args_for_lda)

                    # solve instance with Gurobi
                    args_for_gurobi = utils.merge_two_dicts(args_grb,args_common)
                    X_grb,cpu_grb = grb.solve(I,T,Y,W,D,p,args_for_gurobi)

                    # compute true objective function values
                    obj_ca = utils.omega(I,T,X_ca,Y,W,D,p,args_common)
                    obj_lda = utils.omega(I,T,X_lda,Y,W,D,p,args_common)
                    obj_grb = 0
                    if X_grb:
                        obj_grb = utils.omega(I,T,X_grb,Y,W,D,p,args_common)
                    # save results to txt file
                    for t in T:
                        grb_x = [-1,-1,-1]
                        if X_grb:
                            grb_x = X_grb[t]
                        txt += '{:^20},{:^10},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f},{:^10.2f}\n'\
                        .format('{}_{}_{}_{}_{}.txt'.format(s,t,trend_w,trend_d,rep_no),
                                t,
                                grb_x[0],
                                grb_x[1],
                                grb_x[2],
                                X_lda[t][0],
                                X_lda[t][1],
                                X_lda[t][2],
                                X_ca[t][0],
                                X_ca[t][1],
                                X_ca[t][2],
                                obj_grb,
                                obj_lda,
                                obj_ca,
                                cpu_grb,
                                cpu_lda,
                                cpu_ca
                               )
                        print(txt)



10
      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   

      Inst

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

      Instance      ,  Period  ,  x_grb   ,  y_grb   ,  h_grb   ,  x_lda   ,  y_lda   ,  h_lda   ,   x_ca   ,   y_ca   ,   h_ca   , obj_grb  , obj_lda  ,  obj_ca  , cpu_grb  , cpu_lda  ,  cpu_ca  
 4_0_inc_inc_0.txt  ,    0     ,   0.00   ,   0.00   ,  50.00   , 1194.00  ,  860.00  ,  407.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_1_inc_inc_0.txt  ,    1     ,   0.00   ,   0.00   ,  50.00   , 1122.00  , 1295.00  ,  453.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_2_inc_inc_0.txt  ,    2     ,   0.00   ,   0.00   ,  50.00   ,  75.00   ,  231.00  ,  345.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_3_inc_inc_0.txt  ,    3     ,   0.00   ,   0.00   ,  50.00   ,  448.00  ,  29.00   ,  393.00  ,   0.00   ,   0.00   ,  100.00  ,  13.70   ,  18.67   ,  14.77   ,   0.00   ,   0.13   ,   0.38   
 4_4_inc_inc_0.

KeyboardInterrupt: 

In [ ]:
print(txt)